## 3. Implement an image classification service

Let’s jump right in and get a simple ML service up and running on Ray Serve. 

Here is an image classification service that performs inference on a batch of handwritten digits using an `MNISTClassifier` model.

In [ ]:
class MNISTClassifier:
    def __init__(self, remote_path: str, local_path: str, device: str):
        subprocess.run(f"aws s3 cp {remote_path} {local_path} --no-sign-request", shell=True, check=True)
        
        self.device = device
        self.model = torch.jit.load(local_path).to(device).eval()

    def __call__(self, batch: dict[str, np.ndarray]) -> dict[str, np.ndarray]:
        return self.predict(batch)
    
    def predict(self, batch: dict[str, np.ndarray]) -> dict[str, np.ndarray]:
        images = torch.tensor(batch["image"]).float().to(self.device)

        with torch.no_grad():
            logits = self.model(images).cpu().numpy()

        batch["predicted_label"] = np.argmax(logits, axis=1)
        return batch

First we need to load the classifier model

In [ ]:
model_path = "/mnt/cluster_storage/model.pt" # Use your local path
classifier = MNISTClassifier(remote_path="s3://anyscale-public-materials/ray-ai-libraries/mnist/model/model.pt", local_path=model_path, device="cpu")

Then we can run inference to generate predicted labels

In [ ]:
output = classifier({"image": np.random.rand(1, 1, 28, 28).astype(np.float32)})  # Example input (B, C, H, W)
output["predicted_label"]  # Should be a numpy array with the predicted label

Now, if want to migrate to an online inference setting, we can transform this into a Ray Serve Deployment by applying the `@serve.deployment` decorator 


In [ ]:
@serve.deployment() # this is the decorator to add
class OnlineMNISTClassifier:
    # same code as MNISTClassifier.__init__
    def __init__(self, remote_path: str, local_path: str, device: str):
        subprocess.run(f"aws s3 cp {remote_path} {local_path} --no-sign-request", shell=True, check=True)
        
        self.device = device
        self.model = torch.jit.load(local_path).to(device).eval()

    async def __call__(self, request: Request) -> dict[str, Any]:  # __call__ now takes a Request object
        batch = json.loads(await request.json()) # we will need to parse the JSON body of the request
        return await self.predict(batch)
    
    # same code as MNISTClassifier.predict
    async def predict(self, batch: dict[str, np.ndarray]) -> dict[str, np.ndarray]:
        images = torch.tensor(batch["image"]).float().to(self.device)

        with torch.no_grad():
            logits = self.model(images).cpu().numpy()

        batch["predicted_label"] = np.argmax(logits, axis=1)
        return batch

We have now defined our Ray Serve deployment

In [ ]:
OnlineMNISTClassifier

We can now build an Application using `OnlineMNISTClassifier` deployment

In [ ]:
model_path = "/mnt/cluster_storage/model.pt" # Use your local path
mnist_app = OnlineMNISTClassifier.bind(remote_path="s3://anyscale-public-materials/ray-ai-libraries/mnist/model/model.pt", local_path=model_path, device="cpu")
mnist_app

<div class="alert alert-block alert-info">

**Note:** `.bind` is a method that takes in the arguments to pass to the Deployment constructor.

</div>


We can then run the application 

In [ ]:
mnist_app_handle = serve.run(mnist_app, name='mnist_classifier', blocking=False)
mnist_app_handle

We can test it as an HTTP endpoint

In [ ]:
images = np.random.rand(2, 1, 28, 28).tolist()
json_request = json.dumps({"image": images})
response = requests.post("http://localhost:8000/", json=json_request)
response.json()["predicted_label"]

We can also test it as a gRPC endpoint

In [ ]:
batch = {"image": np.random.rand(10, 1, 28, 28)}
response = await mnist_app_handle.predict.remote(batch)
response["predicted_label"]